# Not Pipeline

In [11]:
from transformers import BertForMaskedLM, BertTokenizerFast, AutoTokenizer, AutoModelForMaskedLM, pipeline
import torch

In [12]:
# Load the fine-tuned model
model = AutoModelForMaskedLM.from_pretrained("ai-forever/ruBert-large")
model.load_state_dict(torch.load('model/bert_best_model.pth'))

C:\Users\robst\anaconda3\envs\RCCKI_W2V_Parser\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at ai-forever/ruBert-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

<All keys matched successfully>

In [13]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-large")

In [14]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [15]:
special_tokens = ["[CLS]", "[MASK]", "[PAD]", "[SEP]", "[UNK]"]

In [16]:
def predict_tokens(sentence, amount = 30):
    # Encode the sentence and find the position of the [MASK] token
    inputs = tokenizer(sentence, return_tensors='pt', padding='max_length', max_length=64, truncation=True)
    mask_pos = torch.where(inputs.input_ids[0] == tokenizer.mask_token_id)[0].tolist()
    # Move tensors to the same device as the model
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    # Get the model's predictions
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = outputs.logits
    # Get the top k predicted token IDs and convert them to tokens
    top_k_predicted_token_ids = torch.topk(predictions[0, mask_pos, :], amount).indices.tolist()
    
    print(torch.topk(predictions[0, mask_pos, :], amount).indices.tolist())
    
    # Flatten the list of lists
    flat_predicted_token_ids = [item for sublist in top_k_predicted_token_ids for item in sublist]
    
    # Convert the token IDs to tokens
    top_k_predicted_tokens = tokenizer.convert_ids_to_tokens(flat_predicted_token_ids)
    
    predicted_tokens = [token for token in top_k_predicted_tokens if token not in special_tokens]
    
    return predicted_tokens
    

In [17]:
# Example sentence with a masked token
sentence1 = "следствие просило суд об [MASK]"
sentence2 = "следствие ходатайствовало об [MASK] перед судом"

x = set(predict_tokens(sentence1))
y = set(predict_tokens(sentence2))

predicted_intersect = x.intersection(y)
print(f"Predictions for sentence 1: {sentence1}\n{x}")
print(f"Predictions for sentence 2: {sentence2}\n{y}")
print(f"Predictions intersect:\n {predicted_intersect}")

for predicted_token in predicted_intersect:
    print(f"{sentence1.replace('MASK', predicted_token)} --- {sentence2.replace('MASK', predicted_token)}")

[[27219, 23284, 23708, 48313, 107515, 85191, 28608, 100, 17612, 90407, 75877, 14766, 44807, 15335, 28170, 31302, 7429, 44813, 100589, 56874, 30134, 49190, 39558, 33710, 35149, 62997, 49278, 4252, 49667, 32857]]
[[27219, 100, 7429, 10136, 16775, 75199, 90407, 44813, 15335, 3034, 42588, 19890, 9351, 36694, 32600, 103634, 30983, 14766, 61537, 13035, 119293, 14627, 1019, 14114, 72199, 75877, 42833, 23708, 17934, 50355]]
Predictions for sentence 1: следствие просило суд об [MASK]
{'отставке', 'увеличении', 'ответственности', 'аресте', 'изменении', 'изоляции', 'отсрочке', 'экстрадиции', 'удалении', 'усилении', 'исключении', 'отводе', 'освобождении', 'избрании', 'увольнении', 'отстранении', 'ужесточении', 'условиях', 'отмене', 'утверждении', 'исполнении', 'ограничении', 'остановке', 'обеспечении', 'экспертизе', 'уничтожении', 'амнистии', 'оценке', 'удовлетворении'}
Predictions for sentence 2: следствие ходатайствовало об [MASK] перед судом
{'обстановке', 'организации', 'ответственности', 'аре

# Pipeline

In [18]:
# Create the pipeline with top_k set to 30
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer, top_k=30)

In [19]:
def create_token_dict(result):
    token_dict = {}
    for token in result:
        if token["token_str"] not in special_tokens:
            token_dict[token["token"]] = {
                "token_str": token["token_str"],
                "token_score": token["score"],
                "token_sequence": token["sequence"]
            }
        else:
            continue
    return token_dict

In [20]:
sentence1_result = create_token_dict(fill_mask(sentence1))
sentence2_result = create_token_dict(fill_mask(sentence2))

intersection = set(sentence1_result.keys()).intersection(set(sentence2_result.keys()))

for token in intersection:
    print(f"{sentence1_result[token]['token_sequence']}({sentence1_result[token]['token_str']}:{sentence1_result[token]['token_score']}) --- {sentence2_result[token]['token_sequence']}({sentence2_result[token]['token_str']}:{sentence2_result[token]['token_score']})")

следствие просило суд об ответственности(ответственности:0.0001128696821979247) --- следствие ходатаиствовало об ответственности перед судом(ответственности:0.0408891998231411)
следствие просило суд об исключении(исключении:0.00023277508444152772) --- следствие ходатаиствовало об исключении перед судом(исключении:0.0019046157831326127)
следствие просило суд об экспертизе(экспертизе:0.0004230121267028153) --- следствие ходатаиствовало об экспертизе перед судом(экспертизе:0.008529109880328178)
следствие просило суд об исполнении(исполнении:0.0001663137081777677) --- следствие ходатаиствовало об исполнении перед судом(исполнении:0.007433668244630098)
следствие просило суд об избрании(избрании:9.460232831770554e-05) --- следствие ходатаиствовало об избрании перед судом(избрании:0.007534338627010584)
следствие просило суд об оценке(оценке:0.00019313418306410313) --- следствие ходатаиствовало об оценке перед судом(оценке:0.0037572649307549)
следствие просило суд об аресте(аресте:0.9855115413